In [1]:
import pickle
import networkx as nx
import numpy as np

In [2]:
with open('GOT-characters-raw.pickle','rb') as fp:
    characters = pickle.load(fp)
len(characters.keys())

340

In [3]:
entities = set()
for chapter in characters.values():
    for entity in chapter:
        name = entity[0]
        entities.add(name)
print('analysis contains',len(entities),'individual entities')

analysis contains 5522 individual entities


In [11]:
characters.keys()

dict_keys(['1–0-Prologue', '1–1-Bran', '1–2-Catelyn', '1–3-Daenerys', '1–4-Eddard', '1–5-Jon', '1–6-Catelyn', '1–7-Arya', '1–8-Bran', '1–9-Tyrion', '1–10-Jon', '1–11-Daenerys', '1–12-Eddard', '1–13-Tyrion', '1–14-Catelyn', '1–15-Sansa', '1–16-Eddard', '1–17-Bran', '1–18-Catelyn', '1–19-Jon', '1–20-Eddard', '1–21-Tyrion', '1–22-Arya', '1–23-Daenerys', '1–24-Bran', '1–25-Eddard', '1–26-Jon', '1–27-Eddard', '1–28-Catelyn', '1–29-Sansa', '1–30-Eddard', '1–31-Tyrion', '1–32-Arya', '1–33-Eddard', '1–34-Catelyn', '1–35-Eddard', '1–36-Daenerys', '1–37-Bran', '1–38-Tyrion', '1–39-Eddard', '1–40-Catelyn', '1–41-Jon', '1–42-Tyrion', '1–43-Eddard', '1–44-Sansa', '1–45-Eddard', '1–46-Daenerys', '1–47-Eddard', '1–48-Jon', '1–49-Eddard', '1–50-Arya', '1–51-Sansa', '1–52-Jon', '1–53-Bran', '1–54-Daenerys', '1–55-Catelyn', '1–56-Tyrion', '1–57-Sansa', '1–58-Eddard', '1–59-Catelyn', '1–60-Jon', '1–61-Daenerys', '1–62-Tyrion', '1–63-Catelyn', '1–64-Daenerys', '1–65-Arya', '1–66-Bran', '1–67-Sansa', '1–68

In [15]:
with open('awoif.pkl','rb') as fp:
    awoif = pickle.load(fp)
awoif = awoif.to_dict(orient='index')
awoif

{0: {'aliases': None,
  'books': [3, 4, 0],
  'fullname': None,
  'infoboxSize': 7,
  'links': [],
  'name': 'Sumner Crakehall',
  'pageSize': 8821,
  'roleInBooks': ['mentioned', 'mentioned'],
  'roleScore': 1.0,
  'titles': ['Lord of Crakehall'],
  'score': 0.1596291817279482},
 1: {'aliases': ['Lamprey'],
  'books': [3, 0],
  'fullname': None,
  'infoboxSize': 4,
  'links': [],
  'name': None,
  'pageSize': 8206,
  'roleInBooks': ['appears'],
  'roleScore': 0.8,
  'titles': None,
  'score': 0.11456801446565676},
 2: {'aliases': ['The stallion who mounts the world'],
  'books': [1, 2, 3, 4, 0],
  'fullname': None,
  'infoboxSize': 9,
  'links': [],
  'name': 'Rhaego',
  'pageSize': 10887,
  'roleInBooks': ['mentioned', 'mentioned', 'mentioned', 'mentioned'],
  'roleScore': 2.0,
  'titles': None,
  'score': 0.32570965066776125},
 3: {'aliases': None,
  'books': [0, 0],
  'fullname': None,
  'infoboxSize': 4,
  'links': [],
  'name': 'Jeyne Waters',
  'pageSize': 8940,
  'roleInBooks':

In [20]:
def getMVP(name, shortened, key, equal=True, title=False):
    candidates = []
        
    for s in shortened:
        if key != 'aliases':
            if equal and s[key] is not None:
                if not title:
                    if name == s[key]:
                        candidates.append(s)
                elif s['titles'] is not None:
                    for t in s['titles']:
                        if name == t + ' ' + s[key]:
                            candidates.append(s)
                        
            elif not equal and s[key] is not None:
                if not title:
                    if name in s[key]:
                        candidates.append(s)
                elif s['titles'] is not None:
                    for t in s['titles']:
                        if name in t + ' ' + s[key]:
                            candidates.append(s)
        else:
            if equal and s['aliases'] is not None:
                for a in s['aliases']:
                    if name == a:
                        candidates.append(s)
            elif not equal and s['aliases'] is not None:
                for a in s['aliases']:
                    if name in a:
                        candidates.append(s)
    # at equal level, we give priority to lenghty wikipedia (normalize) * links (normalize)
    if len(candidates)>0:
        mvp = sorted(candidates, key=lambda c: c['score'], reverse=True)[0]
        return mvp['name']
    else:
        None

def lemmatize(name, book):
    shortened = []
    for c in awoif.values():
        if c['books'] is not None:
            if book in c['books']:
                shortened.append(c)
    
    priority_fields = ['name', 'fullname', 'aliases']
    priority_equality = [True, False]
    priority_title = [False, True]
    
    for pf in priority_fields:
        for pe in priority_equality:
            for pt in priority_title:
                candidate = getMVP(name, shortened, pf, pe, pt)
                if candidate is not None:
                    return candidate
    return None


print(lemmatize('Bran',1))
print(lemmatize('Ned',1))
print(lemmatize('Ser Jorah',1))


Brandon Stark
Eddard Stark
Jorah Mormont


In [31]:
def get_edges_from_chapter(chapter,book):
    chapter.sort(key=lambda a:a[1],reverse=False)
    
    chapter_clean = []
    for i in range(0,len(chapter)):
        name1,start1,end1 = chapter[i]
        name1 = lemmatize(name1,book)
        chapter_clean.append((name1,start1,end1))
        
    potential_edges = []
    for i in range(0,len(chapter_clean)):
        name1,start1,end1 = chapter_clean[i]
        for j in range(i+1,len(chapter_clean)):    
            name2,start2,end2 = chapter_clean[j]
            
            if name2 != name1:
                potential_edges.append((name1,name2,{'dist':start2-end1}))

    edges = {}
    for name1,name2,value in potential_edges:
        if (name1,name2) not in edges.keys():
            if (name2,name1) not in edges.keys():
                edges[(name1,name2)] = {'values':[value['dist']]}
            else:
                edges[(name2,name1)]['values'].append(value['dist'])
        else:
            edges[(name1,name2)]['values'].append(value['dist'])

    for k,v in edges.items():
        #edges[k]['weight']=np.percentile(v['values'],10)
        # the 448 value is the mean + standard deviation of the length of a paragraph in game of thrones books 
        edges[k]['weight']=len([a for a in v['values'] if a <448])
    
    ready_for_nx = []
    for k,v in edges.items():
        name1 = k[0]
        name2 = k[1]
        weight = v['weight']
        if weight > 0:
            ready_for_nx.append((name1,name2,{'weight':weight}))
    return ready_for_nx

In [32]:
characters['5–54-Cersei']

[('Kingsguard', 1211, 1221),
 ('Taena', 2871, 2876),
 ('Lancel', 3183, 3189),
 ('Prayer', 4841, 4847),
 ('Crone', 5499, 5504),
 ('Stranger', 5547, 5555),
 ('Night', 6367, 6372),
 ('Crone', 8535, 8540),
 ('Grace', 9635, 9640),
 ('Hush', 10906, 10910),
 ('Crone', 11963, 11968),
 ('Unella', 12030, 12036),
 ('Lancel Lannister', 12792, 12808),
 ('Safer', 13011, 13016),
 ('Kingsguard', 13833, 13843),
 ('Kingsguard', 13849, 13859),
 ('High Holiness', 14087, 14100),
 ('Forgiveness', 14214, 14225),
 ('Cersei', 14391, 14397),
 ('Lancel', 14436, 14442),
 ('Lancel', 14873, 14879),
 ('Holy Faith', 16398, 16408),
 ('Faith', 25609, 25614),
 ('Kingsguard', 25728, 25738),
 ('Tarly', 26695, 26700),
 ('Dornishmen', 27648, 27658),
 ('Martells', 27680, 27688),
 ('this Ser Gerold', 28536, 28551),
 ('this Gerold Dayne.', 28892, 28910),
 ('Kingsguard', 29500, 29510),
 ('Kingsguard', 29775, 29785),
 ('Cersei', 215, 221),
 ('Jaime', 649, 654),
 ('Ser Loras', 975, 984),
 ('Dragonstone', 1045, 1056),
 ('Cersei', 

In [33]:
GOT = nx.Graph()

In [34]:
for k,v in characters.items():
    print('book',int(k.split('–')[0]),'chapter',int(k.split('–')[1].split('-')[0]))
    GOT.add_edges_from(get_edges_from_chapter(v,int(k.split('–')[0])))

book 1 chapter 0
book 1 chapter 1
book 1 chapter 2
book 1 chapter 3
book 1 chapter 4
book 1 chapter 5
book 1 chapter 6
book 1 chapter 7
book 1 chapter 8
book 1 chapter 9
book 1 chapter 10
book 1 chapter 11
book 1 chapter 12
book 1 chapter 13
book 1 chapter 14
book 1 chapter 15
book 1 chapter 16
book 1 chapter 17
book 1 chapter 18
book 1 chapter 19
book 1 chapter 20
book 1 chapter 21
book 1 chapter 22
book 1 chapter 23
book 1 chapter 24
book 1 chapter 25
book 1 chapter 26
book 1 chapter 27
book 1 chapter 28
book 1 chapter 29
book 1 chapter 30
book 1 chapter 31
book 1 chapter 32
book 1 chapter 33
book 1 chapter 34
book 1 chapter 35
book 1 chapter 36
book 1 chapter 37
book 1 chapter 38
book 1 chapter 39
book 1 chapter 40
book 1 chapter 41
book 1 chapter 42
book 1 chapter 43
book 1 chapter 44
book 1 chapter 45
book 1 chapter 46
book 1 chapter 47
book 1 chapter 48
book 1 chapter 49
book 1 chapter 50
book 1 chapter 51
book 1 chapter 52
book 1 chapter 53
book 1 chapter 54
book 1 chapter 55
bo

In [ ]:
print(nx.info(GOT))

In [85]:
nx.write_gexf(GOT,'GOT-version-2.gexf')